In [ ]:
# Import necessary libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv
import time

# Initialize Chrome driver (in headless mode for faster scraping)
try:
    driver = webdriver.Chrome()
except:
    from selenium.webdriver.chrome.options import Options
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Chrome(options=chrome_options)
    print("Running in headless mode.")

# URL of the page containing the desired links
url = "https://urdu.news18.com/sports/"
driver.get(url)

# Wait for the page to load
wait = WebDriverWait(driver, 10)

# Open the CSV file in append mode
with open(r"C:\Users\anura\Desktop\NLP\Assignment_1\urls.csv", mode="a", newline='', encoding='utf-8') as file:
    writer = csv.writer(file)

    # Infinite loop to keep clicking 'Load More' button until it disappears
    while True:
        try:
            # Find all list items containing links
            list_items = driver.find_elements(By.CSS_SELECTOR, 'li.jsx-bcef60bca6506e85 a')

            for item in list_items:
                link = item.get_attribute('href')
                if link and "/news" in link:  # Filter for links containing "/news"
                    full_link = link if link.startswith("http") else "https://urdu.news18.com" + link
                    print(f"Found URL: {full_link}")
                    writer.writerow([full_link])  # Append the URL to the CSV file

            # Scroll down to load more content (click on 'Load More' button if present)
            load_more_button = wait.until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.jsx-2498664463.load_more'))
            )
            load_more_button.click()  # Click the 'Load More' button
            time.sleep(2)  # Wait for new content to load

        except Exception as e:
            print(f"Finished loading or encountered an error: {str(e)}")
            break  # Exit the loop when there is no 'Load More' button or an error occurs

# Close the browser
driver.quit()

print("Scraping complete. URLs saved to urls.csv.")


In [ ]:
# Import necessary libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv
import time

# Initialize Chrome driver (headless mode for faster scraping)
try:
    driver = webdriver.Chrome()
except:
    from selenium.webdriver.chrome.options import Options
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Chrome(options=chrome_options)
    print("Running in headless mode.")

# Open the CSV file containing URLs (urls.csv)
with open(r"C:\Users\anura\Desktop\NLP\Assignment_1\news_18_education_urls.csv", "r", newline='', encoding='utf-8') as input_file:
    urls_reader = csv.reader(input_file)

    # Open a new CSV file to store extracted articles (news18_sports.csv)
    with open(r"C:\Users\anura\Desktop\NLP\Assignment_1\news18_education.csv", mode="a", newline='', encoding='utf-8') as output_file:
        writer = csv.writer(output_file)

        # Write the header row (URL, Heading, Content)
        writer.writerow(["URL", "Heading", "Content"])

        # Iterate over each URL in the input CSV
        for row in urls_reader:
            url = row[0]  # Get the URL from the first column
            driver.get(url)

            try:
                # Wait for the heading to load and extract the heading text
                heading = WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "#target-div > h1"))
                ).text

                # Keep clicking "Read More" until it's no longer available
                while True:
                    try:
                        read_more_button = WebDriverWait(driver,1).until(
                            EC.element_to_be_clickable((By.ID, "readmorebox"))
                        )
                        read_more_button.click()
                        time.sleep(1)  # Wait for the content to load
                    except:
                        break  # Exit loop when no more "Read More" button

                # Extract all paragraphs inside the main content section
                paragraphs = driver.find_elements(By.CSS_SELECTOR, "#main-content p")
                content = "\n".join([p.text for p in paragraphs])  # Join all paragraphs into one string

                # Write the URL, heading, and full content to the CSV file
                writer.writerow([url, heading, content])
                print(f"Article saved: {url}")

            except Exception as e:
                print(f"Error processing {url}: {str(e)}")

# Close the browser
driver.quit()

print("Article extraction complete. Data saved to news18_sports.csv.")
